In [4]:
################# 실제 작동되는 코드 ####################

import paramiko
import os

ip = "192.168.0.18"
user = "root"
pw = "asd123!@"

SERVER_NFS_DIR = ""
CLIENT_NFS_DIR = ""

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    return out + err
##################################
#          SERVER_NFS            #
##################################

def server_nfs_install():
    cc("apt update; apt install nfs-kernel-server -y")

def server_nfs_mkdir():
    global SERVER_NFS_DIR
    SERVER_NFS_DIR = input("서버가 NFS로 공유해줄 디렉토리를 입력해주세요 ex. /data/nfs_share ")
    cmd =  f"""
mkdir -p {SERVER_NFS_DIR}
chown nobody:nogroup {SERVER_NFS_DIR}
chmod 777 {SERVER_NFS_DIR}
    """
    cc(cmd)

def server_nfs_set_exports_config():
    CLIENT_IP = input("NFS 공유를 허용해줄 클라이언트의 IP를 입력해주세요 ex. 172.16.16.100")
    print(f"클라이언트 CLIENT_IP 에게 기본적으로 읽기 쓰기 권한을 주겠습니다.")
    cc(f"echo '{SERVER_NFS_DIR} {CLIENT_IP}(rw,sync,no_subtree_check)' >> /etc/exports")
    cc("exportfs -a ; systemctl restart nfs-kernel-server")
    cc("systemctl stop ufw")

def server_nfs():
    server_nfs_install()
    server_nfs_mkdir()
    server_nfs_set_exports_config()
##################################
#          CLIENT_NFS            #
##################################

def client_nfs_install():
    cc("apt update ; apt install nfs-common -y")

def client_nfs_mkdir():
    global CLIENT_NFS_DIR
    CLIENT_NFS_DIR = input("클라가 NFS로 공유받을 디렉토리를 입력해주세요 ex. /mnt/nfs_share ")
    cmd =  f"""
mkdir -p {CLIENT_NFS_DIR}
    """
    cc(cmd)
def client_nfs_mount_temporarily():
    SERVER_IP = input("NFS 서버 IP를 입력하세요 (ex. 172.16.16.100): ")
    CLIENT_NFS_DIR = input("클라이언트 마운트 경로를 입력하세요 (ex. /mnt/nfs): ")
    SERVER_NFS_DIR = input("서버 공유 디렉토리 경로를 입력하세요 (ex. /srv/nfs): ")
    
    cmd = f"mount {SERVER_IP}:{SERVER_NFS_DIR} {CLIENT_NFS_DIR}"
    cc(cmd)  # cc는 SSH 또는 로컬 명령 실행 함수

def client_nfs():
#    client_nfs_install()
#    client_nfs_mkdir()
    client_nfs_mount_temporarily()

#print(cc("ls -l"))
#server_nfs()
client_nfs()

NFS 서버 IP를 입력하세요 (ex. 172.16.16.100):  192.168.0.100
클라이언트 마운트 경로를 입력하세요 (ex. /mnt/nfs):  /mnt/nfs
서버 공유 디렉토리 경로를 입력하세요 (ex. /srv/nfs):  /srv/nfs
